In [1]:
from __future__ import division #, print_function # Imports from __future__ since we're running Python 2

In [2]:
#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [95]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
random_state = 10 # Ensure reproducible results
%matplotlib inline
plt.style.use('ggplot')
from flights_delay.feature_processing import FlightDelayFeatureProcessing
from helpers.outliers import MyOutliers

In [4]:
data_relative_path = '../Data'

# Flying to New York City - Raw Data Preprocess

In [5]:
path_data = os.path.realpath(os.path.join(os.getcwd(), data_relative_path, 'raw_data.csv'))
assert os.path.isfile(path_data)
path_data

'/home/student/pligor.george@gmail.com/msc_Artificial_Intelligence/dme_Data_Mining/dmedatarats/Data/raw_data.csv'

In [6]:
df = pd.read_csv(path_data, delimiter = ',', index_col=0, header=0)
df.shape

(492181, 39)

In [7]:
print("There are {} entries and {} columns in the DataFrame"\
      .format(df.shape[0], df.shape[1]))

There are 492181 entries and 39 columns in the DataFrame


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 492181 entries, 0 to 492180
Data columns (total 39 columns):
YEAR                   492181 non-null int64
QUARTER                492181 non-null int64
MONTH                  492181 non-null int64
DAY_OF_MONTH           492181 non-null int64
DAY_OF_WEEK            492181 non-null int64
FL_DATE                492181 non-null object
UNIQUE_CARRIER         492181 non-null object
AIRLINE_ID             492181 non-null int64
CARRIER                492181 non-null object
FL_NUM                 492181 non-null int64
ORIGIN_AIRPORT_ID      492181 non-null int64
ORIGIN                 492181 non-null object
ORIGIN_CITY_NAME       492181 non-null object
ORIGIN_STATE_ABR       492181 non-null object
ORIGIN_STATE_NM        492181 non-null object
DEST_AIRPORT_ID        492181 non-null int64
DEST                   492181 non-null object
DEST_CITY_NAME         492181 non-null object
DEST_STATE_ABR         492181 non-null object
DEST_STATE_NM          4

#### Create is flight delayed boolean

In [9]:
df_1 = FlightDelayFeatureProcessing.createIsDelayedCol(df = df, shuffle_it=False)
df_1.shape

(492181, 40)

#### Remove all arrival columns

In [10]:
df_2 = FlightDelayFeatureProcessing.removeArrivalAttrs(df_1)
df_2.shape

(492181, 34)

#### Remove also elapsed time because it indicates when the airplane arrives, which is what we want to predict

In [11]:
df_3 = FlightDelayFeatureProcessing.removeElapsedTime(df_2)
df_3.shape

(492181, 32)

In [12]:
df_4 = FlightDelayFeatureProcessing.eraseCancelledFlights(df_3)
df_4.shape

(483232, 32)

In [13]:
df_5 = FlightDelayFeatureProcessing.removeCancelledAndFlights(df_4)
df_5.shape

(483232, 30)

#### Erase rows with nulls

In [14]:
df_6 = FlightDelayFeatureProcessing.eraseRowsWithNulls(df_5)
df_6.shape

(483232, 30)

In [15]:
FlightDelayFeatureProcessing.getAllRowsWithAtLeastOneNull(df_6)

set()

### CONCLUSION: ONLY CANCELLED FLIGHTS HAD MISSING DATA, ALL THE REST ARE OK

#### CRS departure time is not a useful information

In [16]:
df_7 = FlightDelayFeatureProcessing.removeCRSDeptTime(df_6)
df_7.shape

(483232, 29)

#### Remove redundant State names

In [17]:
df_8 = FlightDelayFeatureProcessing.removeRedundantStateNames(df_7)
df_8.shape

(483232, 27)

#### Remove Origin Airport Id, Dest Airport Id and Airline Id

In [18]:
df_9 = FlightDelayFeatureProcessing.removeIdColsAlreadyRepresentedByStrCols(df_8)
df_9.shape

(483232, 24)

#### remove carrier because it is already represented in unique carrier

In [19]:
df_10 = FlightDelayFeatureProcessing.removeCarrier(df_9)
df_10.shape

(483232, 23)

#### remove year since we are dealing with only 2016 and also remove flight date

In [20]:
df_11 = FlightDelayFeatureProcessing.dropYearAndDate(df_10)
df_11.shape

(483232, 21)

In [21]:
df_11.dtypes

QUARTER               int64
MONTH                 int64
DAY_OF_MONTH          int64
DAY_OF_WEEK           int64
UNIQUE_CARRIER       object
FL_NUM                int64
ORIGIN               object
ORIGIN_CITY_NAME     object
ORIGIN_STATE_ABR     object
DEST                 object
DEST_CITY_NAME       object
DEST_STATE_ABR       object
DEP_TIME            float64
DEP_DELAY           float64
DEP_DELAY_NEW       float64
DEP_DEL15           float64
DEP_DELAY_GROUP     float64
DEP_TIME_BLK         object
DISTANCE            float64
DISTANCE_GROUP        int64
IS_DELAYED             bool
dtype: object

## Too many instances for our hardward, let's reduce them by taking the some kind of popular destinations and kind of popular origins - kind of popular trips

In [53]:
testdf = df_11.copy()

In [71]:
countPerOrigin = testdf.groupby(['ORIGIN', 'DEST']).size()

In [67]:
len(testdf[np.logical_and(testdf['ORIGIN'] == 'BWI',  testdf['DEST'] == 'ALB')])

1887

In [73]:
countSorted = countPerOrigin.sort_values()[::-1]
countSorted

ORIGIN  DEST
LAX     JFK     12481
JFK     LAX     12476
LGA     ORD      9819
ORD     LGA      9730
SFO     JFK      8020
JFK     SFO      8020
ATL     LGA      8016
LGA     ATL      8008
MIA     LGA      6387
LGA     BOS      6355
BOS     LGA      6333
LGA     MIA      6333
FLL     LGA      5565
LGA     FLL      5554
JFK     BOS      5420
BOS     JFK      5412
JFK     MCO      5405
MCO     JFK      5397
DFW     LGA      4448
LGA     DFW      4442
JFK     FLL      4365
FLL     JFK      4344
LGA     MCO      4324
MCO     LGA      4312
JFK     SJU      4292
SJU     JFK      4283
JFK     LAS      4074
LAS     JFK      4073
LGA     CLT      4072
CLT     LGA      4015
                ...  
ALB     DEN         9
DEN     ALB         9
        BUF         9
BUF     DEN         9
PHL     BUF         8
BZN     LGA         8
LGA     BZN         8
BUF     PHL         8
CVG     JFK         7
JFK     CVG         6
        PHL         5
LGA     MEM         4
SBN     LGA         3
AVL     LGA        

In [82]:
mostPopular = countSorted[:10].copy()
mostPopular

ORIGIN  DEST
LAX     JFK     12481
JFK     LAX     12476
LGA     ORD      9819
ORD     LGA      9730
SFO     JFK      8020
JFK     SFO      8020
ATL     LGA      8016
LGA     ATL      8008
MIA     LGA      6387
LGA     BOS      6355
dtype: int64

In [88]:
popular = pd.DataFrame()

In [89]:
for key in mostPopular.keys():
    print key
    subset = testdf[np.logical_and(testdf['ORIGIN'] == key[0], testdf['DEST'] == key[1])]
    print len(subset)
    print
    popular = pd.concat((
            popular, subset            
        ))

('LAX', 'JFK')
12481

('JFK', 'LAX')
12476

('LGA', 'ORD')
9819

('ORD', 'LGA')
9730

('SFO', 'JFK')
8020

('JFK', 'SFO')
8020

('ATL', 'LGA')
8016

('LGA', 'ATL')
8008

('MIA', 'LGA')
6387

('LGA', 'BOS')
6355



In [90]:
np.sum(mostPopular)

89312

In [91]:
len(popular)

89312

In [93]:
len(popular[popular['IS_DELAYED']])

19449

In [94]:
# fig, ax = plt.subplots(figsize=(16,5))

# sns.barplot(x='ORIGIN', y=countPerOrigin.values, data=df_11, #ax=ax,
#             color = "r")

# plt.show()

### Remove Outliers

In [96]:
popular.dtypes

QUARTER               int64
MONTH                 int64
DAY_OF_MONTH          int64
DAY_OF_WEEK           int64
UNIQUE_CARRIER       object
FL_NUM                int64
ORIGIN               object
ORIGIN_CITY_NAME     object
ORIGIN_STATE_ABR     object
DEST                 object
DEST_CITY_NAME       object
DEST_STATE_ABR       object
DEP_TIME            float64
DEP_DELAY           float64
DEP_DELAY_NEW       float64
DEP_DEL15           float64
DEP_DELAY_GROUP     float64
DEP_TIME_BLK         object
DISTANCE            float64
DISTANCE_GROUP        int64
IS_DELAYED             bool
dtype: object

From all the features only the dep delay and distance make sense to be considered for outliers (we neglect the destination feature for now, even though this could have been considered among the outliers)

# ???????????????????

### Unique values per feature


YEAR: 
[2016]

QUARTER: 
[1 2 3 4]

MONTH: 
[ 1  2  3  4  5  6  7  8  9 10 11 12]

DAY_OF_MONTH: 
[ 4  5  6  7 16 17 18  2 12 13 14 15 25 26 27 21 22  1  3  8  9 10 11 20 19
 30 31 28 29 23 24]

DAY_OF_WEEK: 
[1 2 3 4 6 7 5]

FL_DATE: 
Too large to show

UNIQUE_CARRIER: 
['F9' 'HA' 'DL' 'NK' 'EV' 'OO' 'UA' 'VX' 'AA' 'AS' 'B6' 'WN']

AIRLINE_ID: 
[20436 19690 19790 20416 20366 20304 19977 21171 19805 19930 20409 19393]

CARRIER: 
['F9' 'HA' 'DL' 'NK' 'EV' 'OO' 'UA' 'VX' 'AA' 'AS' 'B6' 'WN']

FL_NUM: 
Too large to show

ORIGIN_AIRPORT_ID: 
[13303 12953 10397 12173 11433 10423 15304 15024 14107 11697 10994 10792
 11292 13204 14747 14683 11298 12892 14869 12889 14027 14576 12478 14679
 13495 10257 10721 14524 14843 14771 13930 13577 12265 12197 14025 15096
 11618 12397 10577 12266 13502 11259 14262 11278 11057 14122 11503 14100
 15070 12451 12954 12191 14831 14254 10732 14685 14492 14057 12264 14635
 10785 14321 10140 14570 14893 14986 13796 10800 11252 13487 10693 10821
 12339 12391 1501

In [ ]:
# clear_data_path = os.path.realpath(os.path.join(os.getcwd(), '../Data', 'clear_data.csv'))
# df_clear.to_csv(clear_data_path, index=False)